# State Construction Task

In [37]:
from datetime import datetime
import itertools
import io

import numpy as np
import scipy.sparse as sp
import tensorflow as tf

from tqdm import tqdm

generates equidistributed points on unit nd-sphere (see [here](https://www.cmu.edu/biolphys/deserno/pdf/sphere_equi.pdf))

In [39]:
def equidistributed_points(dim, dist):
    with np.errstate(divide='ignore'):
        if np.isnan(dist):
            dist = 4 * np.pi
        if dim == 2:
            phis = np.arange(0, 2 * np.pi, dist)
            return np.vstack([np.cos(phis), np.sin(phis)])
        else:
            slices = []
            for phi in np.arange(0, np.pi, dist):
                proj_points = equidistributed_points(dim - 1, dist / np.sin(phi))
                points = np.vstack([np.full((1, proj_points.shape[1]), np.cos(phi)), np.sin(phi) * proj_points])
                slices.append(points)
            return np.hstack(slices)

## solve using Tensorflow

parametric hamiltonian generator with `tf.Variable`

In [40]:
dim = 2

def h_generator():
    def generate_parametric(n, dv_lambda, name):
        params = tf.get_variable(name, dtype='float64', shape=(n, ), initializer=tf.truncated_normal_initializer)
        params = tf.cast(params, 'complex128')
        places = tf.constant(np.stack([dv_lambda(c) for c in range(n)]), dtype='complex128')
        return tf.tensordot(places, params, axes=[0, 0])
    

    neardiag = lambda i : sp.coo_matrix(
        ([1, 1], ([i, (i + 1) % dim], [(i + 1) % dim, i % dim])), shape=(dim, dim)).toarray()

    diag = lambda i : sp.coo_matrix(([1], ([i], [i])), shape=(dim, dim)).toarray()
    h = generate_parametric(dim - 1, neardiag, 'neardiag') + generate_parametric(dim, diag, 'diag')
    
    return h

design own solver

In [44]:
def batch_solve_tensorflow(dim, h_generator, source, targets):
    tf.reset_default_graph()

    source = tf.constant(source, dtype='complex128')
    target = tf.placeholder(dtype='complex128', shape=(dim, ))

    h = h_generator()
    gate = tf.linalg.expm(1j * h)
    estimated_target = tf.matmul(gate, tf.expand_dims(source, 1))

    target_dot = tf.matmul(tf.expand_dims(target, 1), estimated_target, adjoint_a=True)
    fidelity = tf.abs(target_dot)
    loss = 1 - fidelity

    summary_op = tf.summary.scalar('fidelity', fidelity)
    optimize_op = tf.train.MomentumOptimizer(momentum=0.8,learning_rate=0.5).minimize(loss)
    
    fidelities = []
    with tf.Session() as session:
        init_op = tf.global_variables_initializer()
        for t in tqdm(targets):
            session.run(init_op)
            for i in range(5000): # todo, improve stop
                session.run(optimize_op, feed_dict={target: t})
            fidelities.append(session.run(fidelity, feed_dict={target: t}))
    
    return min(fidelities)

In [45]:
angle = 0.1

source = np.array([1] + [0] * (dim - 1))
targets = list(equidistributed_points(dim, angle).T)
min_match = batch_solve_tensorflow(dim, h_generator, source, targets)
MGF = min_match * np.cos(angle)
print(MGF)

100%|██████████| 332/332 [29:46<00:00,  5.38s/it]


array([[0.99463484]])